<a href="https://colab.research.google.com/github/Ertadis2/Deep-Learning-for-Forex/blob/main/LSTM_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')



Tensorflow version 2.8.0
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.14.179.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.14.179.106:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.14.179.106:8470']


In [ ]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.layers.core import Dense, Activation, Dropout
from datetime import datetime
import time
from keras import *
import tensorflow as tf
import os
from tensorflow.keras import optimizers
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
scaler_train  = MinMaxScaler(feature_range=(0,11558820))
scaler_test   = MinMaxScaler(feature_range=(0,525960))
scaler_y_high   = MinMaxScaler()
scaler_y_low    = MinMaxScaler()
scaler_y_close  = MinMaxScaler()

df_train =0
df_test  =0
carpan =1
name ="M1"

def TimeStamps(dataframe_Date):
    dTime = dataframe_Date
    if name == ('Weekly'or 'Daily' or 'Monthly'):
        return pd.DataFrame(array([datetime.strptime((dTime[i]),'%Y %m %d').timestamp() for i in range(len(dTime))]))#%H:%M:%S
    else :
        return pd.DataFrame(array([datetime.strptime((dTime[i]),'%Y %m %d %H %M %S').timestamp() for i in range(len(dTime))]))

import dill
try:
   
    df_train = read_csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Projesi/EURUSD_'+name+'_hazirVeri_train.csv', engine='python')
    df_test  = read_csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Projesi/EURUSD_'+name+'_hazirVeri_test.csv', engine='python')
    df_train['Date'] = TimeStamps(df_train['Date'])
    df_test['Date']  = TimeStamps(df_test['Date'])
    dill.dump_session('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Projesi/'+name+'_dataframes.db')
    """
    dill.load_session('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Projesi/'+name+'_dataframes.db')
     """
except:
    
    df_train = read_csv('EURUSD_'+name+'_hazirVeri_train.csv', engine='python')
    df_test  = read_csv('EURUSD_'+name+'_hazirVeri_test.csv' , engine='python')
    df_train['Date'] = TimeStamps(df_train['Date'])
    df_test['Date']  = TimeStamps(df_test['Date'])
    dill.dump_session(name+'_dataframes.db')
    """
    dill.load_session(name+'_dataframes.db')
    """
#df_train['Date'] =pd.DataFrame(scaler_train.fit_transform(np.array(df_train['Date']).reshape(-1,1)),columns=['Date']) 
#df_test['Date'] =pd.DataFrame(scaler_test.fit_transform(np.array(df_test['Date']).reshape(-1,1)),columns=['Date']) 
#df_train = df_train.round(5)
#df_test = df_test.round(5)

df_train,df_test

INFO:tensorflow:Assets written to: ram://f9e085bb-be8b-4000-a170-37daeb1ffb50/assets


INFO:tensorflow:Assets written to: ram://f9e085bb-be8b-4000-a170-37daeb1ffb50/assets


(           Index          Date     Open     High      Low    Close
 0              0  9.468577e+08  1.00790  1.00790  1.00790  1.00790
 1              1  9.468578e+08  1.00780  1.00800  1.00770  1.00800
 2              2  9.468583e+08  1.00790  1.00790  1.00790  1.00790
 3              3  9.468584e+08  1.00800  1.00800  1.00770  1.00770
 4              4  9.468584e+08  1.00780  1.00790  1.00760  1.00770
 ...          ...           ...      ...      ...      ...      ...
 7989910  7989910  1.640909e+09  1.13245  1.13253  1.13244  1.13253
 7989911  7989911  1.640909e+09  1.13253  1.13254  1.13251  1.13252
 7989912  7989912  1.640909e+09  1.13253  1.13253  1.13246  1.13248
 7989913  7989913  1.640909e+09  1.13253  1.13257  1.13251  1.13256
 7989914  7989914  1.640909e+09  1.13244  1.13244  1.13244  1.13244
 
 [7989915 rows x 6 columns],
        Index          Date     Open     High      Low    Close
 0          0  1.641168e+09  1.13689  1.13690  1.13652  1.13655
 1          1  1.641168e+

In [ ]:
model = Sequential()
giris = 2
cikis = 3
def SetModel(units,train,dense,activations='relu',optimizer='adam', loss='mse'):
    print("Set Mode train shape 0 : ",train.shape[0])
    model.add(LSTM(units, activation=activations,input_shape=(train.shape[1],giris)))
    """
    #model.add(Dropout(0.2))
    model.add(Dense(4))
    model.add(Dense(4))
    model.add(Dense(4))
    model.add(Dense(4))
    #model.add(Dense(512, activation=activations))
    model.add(Dense(dense))
    model.compile(optimizer, loss)
    """
    model.add(Dense(256*4))
    #model.add(Dropout(0.2))
    model.add(Dense(256*2))
    #model.add(Dropout(0.2))
    model.add(Dense(256))
    #model.add(Dropout(0.2))
    model.add(Dense(256))
    #model.add(Dropout(0.2))
    model.add(Dense(128))
    #model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(Dense(cikis))
    opt = tf.keras.optimizers.SGD(learning_rate=1)
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    print(model.summary())

def setupDataFrame(dataFrame):
    #X
    X_Data = pd.DataFrame(dataFrame,columns=['Index','Date','Open','High','Low','Close'])
    X_Data = X_Data.drop('Index',axis=1)
    X_Data = X_Data.drop('High',axis=1)
    X_Data = X_Data.drop('Close',axis=1)
    X_Data = X_Data.drop('Low',axis=1)
    if(giris==1):
        X_Data = X_Data.drop('Date',axis=1)
    #X_Data['Date'] = pd.to_datetime(X_Data['Date'], infer_datetime_format=True)
    #X_Data.set_index('Date', inplace=True)
    #X_Data['Date'] = X_Data['Date'].astype(float)
    #X_Data = np.array(X_Data)
    print("Orjinal Shape X :",X_Data.shape)
    X_Data = np.array(X_Data).reshape((len(X_Data),1,giris))
    print("Yeni Shape X :",X_Data.shape)
    #X_Data[:,1] = scaler_x.fit_transform(X_Data[:,1])#.round(5)

    #Y
    Y_Data = pd.DataFrame(dataFrame,columns=['Index','Date','Open','High','Low','Close'])
    Y_Data = Y_Data.drop('Index',axis=1)
    Y_Data = Y_Data.drop('Date',axis=1)
    if(cikis==3):
        Y_Data = Y_Data.drop('Open',axis=1)#
    Y_Data = (Y_Data*carpan).values#.round(5)
    print("Orjinal Shape Y :",Y_Data.shape)
    Y_Data = np.array(Y_Data).reshape(len(Y_Data),1,cikis)
    #Y_Data = scaler.fit_transform(Y_Data)#.round(5)
    print("Yeni Shape Y :",Y_Data.shape)
    #print(X_t,y_t)
    return X_Data,Y_Data

def FUNC_Time(train_dataframe,test_dataframe):
    #X_train #Y_train
    X_train,y_train =setupDataFrame(train_dataframe)
    #X_test #y_test
    X_test,y_test = setupDataFrame(test_dataframe)
    #SetModel
    SetModel(units=2048,train=X_train,dense=cikis)
    #print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    print(X_train,"\n",y_train,"\n",X_test,"\n",y_test,"\n")
    return X_train,y_train,X_test,y_test

X_train,y_train,X_test,y_test = FUNC_Time(df_train,df_test)

Orjinal Shape X : (7989915, 2)
Yeni Shape X : (7989915, 1, 2)
Orjinal Shape Y : (7989915, 3)
Yeni Shape Y : (7989915, 1, 3)
Orjinal Shape X : (56086, 2)
Yeni Shape X : (56086, 1, 2)
Orjinal Shape Y : (56086, 3)
Yeni Shape Y : (56086, 1, 3)
Set Mode train shape 0 :  7989915
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 2048)              16801792  
                                                                 
 dense_21 (Dense)            (None, 1024)              2098176   
                                                                 
 dense_22 (Dense)            (None, 512)               524800    
                                                                 
 dense_23 (Dense)            (None, 256)               131328    
                                                                 
 dense_24 (Dense)            (None, 256)    

In [ ]:
#es_callback = callbacks.EarlyStopping(monitor='val_loss', patience=3) 
#for i in range(100):
history = model.fit(X_train,y_train, epochs=5000,batch_size=4096)  
predict = model.predict(X_test)#.round(5)
print(predict)



Epoch 1/5000
1951/1951 [==============================] - 514s 262ms/step - loss: 86889235218432.0000 - accuracy: 0.4120
Epoch 2/5000
1951/1951 [==============================] - 518s 266ms/step - loss: 12.5395 - accuracy: 0.3912
Epoch 3/5000
1951/1951 [==============================] - 519s 266ms/step - loss: 35265322156032.0000 - accuracy: 0.4005
Epoch 4/5000
1951/1951 [==============================] - 503s 258ms/step - loss: 1828.8956 - accuracy: 1.0000
Epoch 5/5000
1951/1951 [==============================] - 518s 265ms/step - loss: 43.6190 - accuracy: 1.0000
Epoch 6/5000
1951/1951 [==============================] - 514s 263ms/step - loss: 42.5110 - accuracy: 1.0000
Epoch 7/5000
1951/1951 [==============================] - 514s 264ms/step - loss: 40.4381 - accuracy: 1.0000
Epoch 8/5000
1951/1951 [==============================] - 512s 263ms/step - loss: 35.4605 - accuracy: 1.0000
Epoch 9/5000
1951/1951 [==============================] - 508s 260ms/step - loss: 25.2622 - accuracy: 

KeyboardInterrupt: ignored

In [ ]:
name2 = 'son_M1_2'
#dill.dump_session('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Projesi/'+name2+'_dataframes.db')    

In [ ]:
predict = model.predict(X_test)#.round(5)

In [ ]:
print(predict,"\n",y_test)

[[1.2066803 1.6970239 1.0384823]
 [1.2066803 1.6970239 1.0384823]
 [1.2066803 1.6970239 1.0384823]
 ...
 [1.2066804 1.6970237 1.0384822]
 [1.2066804 1.6970237 1.0384822]
 [1.2066804 1.6970237 1.0384822]] 
 [[[1.1369  1.13652 1.13655]]

 [[1.13678 1.13645 1.13678]]

 [[1.13678 1.13673 1.13675]]

 ...

 [[1.1195  1.1193  1.11932]]

 [[1.11932 1.11914 1.11919]]

 [[1.11911 1.11908 1.11911]]]


In [ ]:
#predict2 = scaler_test.inverse_transform(predict)

#predict = scaler.inverse_transform(predict) 
#y_test  = scaler.inverse_transform(y_test)

#fark2 = predict[:50]-y_test[:50]

#predict = pd.DataFrame(((predict)),columns=['High_Predict','Low_Predict','Close_Predict'])
#y_test  = pd.DataFrame(((y_test)),columns=['High_Reel','Low_Reel','Close_Reel'])
#X_test  = pd.DataFrame(((X_test)),columns=['Open'])


#found  = pd.concat([X_test,predict],axis=1)
#expect = pd.concat([X_test,y_test],axis=1)
#print((predict[i]).round(5),(y_test[i]).round(5))
#print(fark2,"\n \n",expect,"\n \n",found)


In [ ]:
print(predict[0,0])

In [ ]:
history.history